Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

# AML ワークスペースのセットアップ
---

このノートブックでは、このソリューション アクセラレータの環境を構成するために必要なすべての手順について説明します:

1. ワークスペースへの接続
2. config.json の作成します (AML コンピューティング インスタンスで実行している場合はスキップできます)
3. コンピューティング クラスターの作成

## 1.0 ワークスペースへの接続

このソリューション アクセラレータを AML ワークスペースに接続します。この手順は、コンピューティング インスタンス（Notebook VM）を使用している場合は必要ありません。

次のセルを使用すると、ワークスペースのパラメータを指定できます。このセルは、python メソッド os.getenv を使用して、オートメーションに役立つ値を環境変数から読み取ります。環境変数が存在しない場合、パラメータは指定されたデフォルト値に設定されます。

In [ ]:
import os

subscription_id = os.getenv("SUBSCRIPTION_ID", default="<my-subscription-id>")
resource_group = os.getenv("RESOURCE_GROUP", default="<my-resource-group>")
workspace_name = os.getenv("WORKSPACE_NAME", default="<my-workspace-name>")
workspace_region = os.getenv("WORKSPACE_REGION", default="westus2")

In [ ]:
from azureml.core import Workspace

try:
    ws = Workspace(subscription_id=subscription_id, 
                   resource_group=resource_group, 
                   workspace_name=workspace_name)
    print("ワークスペースの構成に成功しました。以下のワークスペース作成手順をスキップします")
except:
    print("ワークスペースが存在しません。ワークスペースを作成します")
    ws = Workspace.create(name=workspace_name, subscription_id=subscription_id, resource_group=resource_group,
                            location=workspace_region, create_resource_group=True, sku='enterprise', exist_ok=True)

In [ ]:
ws.get_details()

## 2.0 構成ファイルへの書き込み
ワークスペースの詳細を config.json ファイルに書き込みます:

In [ ]:
ws.write_config()

## 3.0 コンピューティング クラスターの作成

トレーニングおよびバッチ予測用のコンピューティング クラスターが必要です。これは 1 回限りのセットアップなので、今後のノートブックで再実行する必要はありませんが、次のノートブックでは同じクラスター名を使用してこのクラスターを参照する必要があります。

In [ ]:
# CPU クラスタ名を命名
cpu_cluster_name = 'cpucluster'

STANDARD_D13_V2 コンピュート クラスターを作成します。D シリーズ VM は、高いコンピューティング能力と一時的なディスク パフォーマンスを必要とするタスクに使用されます。この[ページ](https://docs.microsoft.com/ja-jp/azure/virtual-machines/sizes-general)では、ユース ケースに最適な方法を決定するために、VM のサイズに関する詳細を説明します。

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# クラスタが存在していないことを確認
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('既存のクラスターを検出したため、代わりにこのクラスターを指定しました。')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D13_V2',
                                                           min_nodes=0,
                                                           max_nodes=20)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)
    cpu_cluster.wait_for_completion(show_output=True)

## 次のステップ

以上でAML ワークスペースの構成が完了しました。次に [01_Data_Preparation.ipynb](01_Data_Preparation.ipynb) の手順に従います。